In [33]:
import h2o

In [35]:
h2o.init()
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,10 mins 31 secs
H2O cluster timezone:,America/Santiago
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,7 months and 4 days !!!
H2O cluster name:,H2O_from_python_maravenag_r26r3s
H2O cluster total nodes:,1
H2O cluster free memory:,3.221 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [36]:
train = h2o.upload_file("data/train.csv", destination_frame="train")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [37]:
train.head(3)
#test no tiene la clases :o

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,nan,7.25,,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,nan,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,nan,7.925,,S


In [38]:
train = train.drop("PassengerId")
train = train.drop("Name")
train = train.drop("Cabin")
train = train.drop("Ticket")

In [39]:
train, test = train.split_frame(ratios=[0.7])

In [40]:
train.head()

Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22,1,0,7.25,S
1,1,female,38,1,0,71.2833,C
1,3,female,26,0,0,7.925,S
1,1,female,35,1,0,53.1,S
0,1,male,54,0,0,51.8625,S
0,3,male,2,3,1,21.075,S
1,3,female,27,0,2,11.1333,S
1,2,female,14,1,0,30.0708,C
1,3,female,4,1,1,16.7,S
0,3,male,20,0,0,8.05,S


In [41]:
train['Survived'] = train['Survived'].asfactor()
train['Pclass'] = train['Pclass'].asfactor()
train['Sex'] = train['Sex'].asfactor()
train['Embarked'] = train['Embarked'].asfactor()

test['Survived'] = test['Survived'].asfactor()
test['Pclass'] = test['Pclass'].asfactor()
test['Sex'] = test['Sex'].asfactor()
test['Embarked'] = test['Embarked'].asfactor()

#### Correr un autoML como baseline

In [42]:
from h2o.automl import H2OAutoML

In [43]:
target = "Survived"

In [44]:
train_cols = [x for x in train.col_names if x not in ['Survived']]

In [45]:
aml = H2OAutoML(max_runtime_secs = 100)
aml.train(x = train_cols, y = target,
          training_frame = train)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [46]:
lb = aml.leaderboard
print(lb)

model_id,auc,logloss
GBM_grid_0_AutoML_20181010_164706_model_14,0.867024,0.413635
StackedEnsemble_AllModels_0_AutoML_20181010_164706,0.866573,0.411107
GBM_grid_0_AutoML_20181010_164706_model_0,0.863811,0.423598
GBM_grid_0_AutoML_20181010_164706_model_3,0.863544,0.419243
StackedEnsemble_BestOfFamily_0_AutoML_20181010_164706,0.863268,0.41305
GBM_grid_0_AutoML_20181010_164706_model_2,0.85973,0.429969
GBM_grid_0_AutoML_20181010_164706_model_5,0.85862,0.523054
XRT_0_AutoML_20181010_164706,0.858603,0.500686
GBM_grid_0_AutoML_20181010_164706_model_1,0.856617,0.426647
GBM_grid_0_AutoML_20181010_164706_model_6,0.853922,0.436287


#### Ocupando un Autoencoder

In [47]:
from h2o.estimators import H2OAutoEncoderEstimator

In [53]:
autoencoder = H2OAutoEncoderEstimator(activation="Tanh",
                              hidden=[10],
                              ignore_const_cols=False,
                              epochs=1000)

In [54]:
autoencoder.train(x=train_cols, training_frame=train)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [55]:
autoencoder._model_json["output"]["training_metrics"]


ModelMetricsAutoEncoder: deeplearning
** Reported on train data. **

MSE: 9.907751523481976e-05
RMSE: 0.009953768895992098


In [56]:
train_deep_feats = autoencoder.predict(train)
train_deep_feats = train_deep_feats.cbind(train['Survived'])
deep_cols = [x for x in train_deep_feats.col_names if x not in ['Survived']]

deeplearning prediction progress: |███████████████████████████████████████| 100%


In [57]:
train_deep_feats.head(5)

reconstr_Pclass.1,reconstr_Pclass.2,reconstr_Pclass.3,reconstr_Pclass.missing(NA),reconstr_Embarked.C,reconstr_Embarked.Q,reconstr_Embarked.S,reconstr_Embarked.missing(NA),reconstr_Sex.female,reconstr_Sex.male,reconstr_Sex.missing(NA),reconstr_Age,reconstr_SibSp,reconstr_Parch,reconstr_Fare,Survived
-0.00210315,-0.00083475,0.999963,3.02864e-05,0.00632466,-0.00228214,0.999959,-0.000443918,0.00654024,0.999966,5.84124e-05,21.615,0.986042,0.00202164,5.32051,0
0.999888,0.00997586,-0.000783716,-8.98257e-05,0.999452,-0.00401009,0.0074651,0.00373011,0.999883,-0.00984709,6.45369e-05,37.6701,1.04611,0.105329,69.7135,1
0.000725499,0.00189306,0.999964,-1.31737e-05,0.000538848,-0.00169568,0.999959,0.000511266,0.999881,-0.0119323,1.34342e-05,26.0363,0.0315288,-0.00520187,10.243,1
0.999967,0.00111342,-0.00218641,5.16592e-05,-0.00269434,-0.0019785,0.999963,-0.00146157,0.999882,0.0129057,0.00054469,35.3663,1.00044,-0.0104004,52.8199,1
0.999967,-0.00105024,0.00399298,-4.0018e-05,-0.000361405,-0.00274086,0.999962,0.000589184,-0.00173386,0.999966,0.000171051,54.1305,-0.0194887,0.0259139,50.7752,0


In [58]:
deep_cols

['reconstr_Pclass.1',
 'reconstr_Pclass.2',
 'reconstr_Pclass.3',
 'reconstr_Pclass.missing(NA)',
 'reconstr_Embarked.C',
 'reconstr_Embarked.Q',
 'reconstr_Embarked.S',
 'reconstr_Embarked.missing(NA)',
 'reconstr_Sex.female',
 'reconstr_Sex.male',
 'reconstr_Sex.missing(NA)',
 'reconstr_Age',
 'reconstr_SibSp',
 'reconstr_Parch',
 'reconstr_Fare']

In [59]:
aml2 = H2OAutoML(max_runtime_secs = 100)
aml2.train(x = deep_cols, y = target,
          training_frame = train_deep_feats)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [60]:
lb = aml2.leaderboard
print(lb)

model_id,auc,logloss
StackedEnsemble_BestOfFamily_0_AutoML_20181010_165048,0.873951,0.410339
GBM_grid_0_AutoML_20181010_165048_model_7,0.87355,0.406645
GBM_grid_0_AutoML_20181010_165048_model_3,0.873166,0.417654
GBM_grid_0_AutoML_20181010_165048_model_1,0.871747,0.416754
StackedEnsemble_AllModels_0_AutoML_20181010_165048,0.869152,0.416793
GBM_grid_0_AutoML_20181010_165048_model_15,0.863727,0.426621
GBM_grid_0_AutoML_20181010_165048_model_2,0.86351,0.427173
XRT_0_AutoML_20181010_165048,0.863302,0.753715
GBM_grid_0_AutoML_20181010_165048_model_0,0.860373,0.442919
GBM_grid_0_AutoML_20181010_165048_model_11,0.855582,0.54175
